Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/training/train-on-amlcompute/train-on-amlcompute.png)

#  Train using Azure Machine Learning Compute

* Initialize a Workspace
* Create an Experiment
* Introduction to AmlCompute
* Submit an AmlCompute run in a few different ways
    - Provision as a run based compute target 
    - Provision as a persistent compute target (Basic)
    - Provision as a persistent compute target (Advanced)
* Additional operations to perform on AmlCompute
* Find the best model in the run

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set.  Otherwise, go through the [configuration](../../../configuration.ipynb) Notebook first if you haven't already to establish your connection to the AzureML Workspace.

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.60


## Initialize a Workspace

Initialize a workspace object from persisted configuration

In [3]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

avadevitsmlsvc
RG-ITSMLTeam-Dev
westus2
ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e


## Create An Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [4]:
from azureml.core import Experiment
experiment_name = 'train-on-amlcompute'
experiment = Experiment(workspace = ws, name = experiment_name)

## Introduction to AmlCompute

Azure Machine Learning Compute is managed compute infrastructure that allows the user to easily create single to multi-node compute of the appropriate VM Family. It is created **within your workspace region** and is a resource that can be used by other users in your workspace. It autoscales by default to the max_nodes, when a job is submitted, and executes in a containerized environment packaging the dependencies as specified by the user. 

Since it is managed compute, job scheduling and cluster management are handled internally by Azure Machine Learning service. 

For more information on Azure Machine Learning Compute, please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)

If you are an existing BatchAI customer who is migrating to Azure Machine Learning, please read [this article](https://aka.ms/batchai-retirement)

**Note**: As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.


The training script `train.py` is already created for you. Let's have a look.

## Submit an AmlCompute run in a few different ways

First lets check which VM families are available in your region. Azure is a regional service and some specialized SKUs (especially GPUs) are only available in certain regions. Since AmlCompute is created in the region of your workspace, we will use the supported_vms () function to see if the VM family we want to use ('STANDARD_D2_V2') is supported.

You can also pass a different region to check availability and then re-create your workspace in that region through the [configuration notebook](../../../configuration.ipynb)

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute

AmlCompute.supported_vmsizes(workspace = ws)
#AmlCompute.supported_vmsizes(workspace = ws, location='southcentralus')

[{'name': 'Standard_F2s_v2', 'vCPUs': 2, 'memoryGB': 4.0},
 {'name': 'Standard_F4s_v2', 'vCPUs': 4, 'memoryGB': 8.0},
 {'name': 'Standard_F8s_v2', 'vCPUs': 8, 'memoryGB': 16.0},
 {'name': 'Standard_F16s_v2', 'vCPUs': 16, 'memoryGB': 32.0},
 {'name': 'Standard_F32s_v2', 'vCPUs': 32, 'memoryGB': 64.0},
 {'name': 'Standard_F64s_v2', 'vCPUs': 64, 'memoryGB': 128.0},
 {'name': 'Standard_F72s_v2', 'vCPUs': 72, 'memoryGB': 144.0},
 {'name': 'Standard_D1_v2', 'vCPUs': 1, 'memoryGB': 3.5},
 {'name': 'Standard_D2_v2', 'vCPUs': 2, 'memoryGB': 7.0},
 {'name': 'Standard_D3_v2', 'vCPUs': 4, 'memoryGB': 14.0},
 {'name': 'Standard_D4_v2', 'vCPUs': 8, 'memoryGB': 28.0},
 {'name': 'Standard_D11_v2', 'vCPUs': 2, 'memoryGB': 14.0},
 {'name': 'Standard_D12_v2', 'vCPUs': 4, 'memoryGB': 28.0},
 {'name': 'Standard_D13_v2', 'vCPUs': 8, 'memoryGB': 56.0},
 {'name': 'Standard_D14_v2', 'vCPUs': 16, 'memoryGB': 112.0},
 {'name': 'Standard_DS1_v2', 'vCPUs': 1, 'memoryGB': 3.5},
 {'name': 'Standard_DS2_v2', 'vCPUs':

### Create project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on

In [6]:
import os
import shutil

project_folder = './train-on-amlcompute'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train.py', project_folder)

'./train-on-amlcompute\\train.py'

### Create environment

Create Docker based environment with scikit-learn installed.

In [9]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment("myenv")

myenv.docker.enabled = True
myenv.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

### Provision as a persistent compute target (Basic)

You can provision a persistent AmlCompute resource by simply defining two parameters thanks to smart defaults. By default it autoscales from 0 nodes and provisions dedicated VMs to run your job in a container. This is useful when you want to continously re-use the same target, debug it between jobs or simply share the resource with other users of your workspace.

* `vm_size`: VM family of the nodes provisioned by AmlCompute. Simply choose from the supported_vmsizes() above
* `max_nodes`: Maximum nodes to autoscale to while running a job on AmlCompute

In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


### Configure & Run

In [11]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

src = ScriptRunConfig(source_directory=project_folder, script='train.py')

# Set compute target to the one created in previous step
src.run_config.target = cpu_cluster.name

# Set environment
src.run_config.environment = myenv

In [ ]:
run = experiment.submit(config=src)
run

In [20]:
from azureml.train.estimator import Estimator
test_estimator = Estimator(
    source_directory=project_folder,
    entry_script="spark_test.py",
    script_params={},
    compute_target=cpu_cluster,
    conda_packages=['pyspark', 'pandas', 'numpy'],
    pip_packages=['pip', 'findspark'],
#     custom_docker_image='microsoft/mmlspark:0.16'
    custom_docker_image='microsoft/mmlspark:latest'
)

WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.
WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.


In [21]:
run = experiment.submit(config=test_estimator)
run

WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.


Experiment,Id,Type,Status,Details Page,Docs Page
train-on-amlcompute,train-on-amlcompute_1568091533_7bb452af,azureml.scriptrun,Starting,Link to Azure Portal,Link to Documentation


In [22]:
from azureml.widgets import RunDetails  # noqa: F401
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Note: if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

In [ ]:
%%time
# Shows output of the run on stdout.
run.wait_for_completion(show_output=True)

In [ ]:
run.get_metrics()

### Provision as a persistent compute target (Advanced)

You can also specify additional properties or change defaults while provisioning AmlCompute using a more advanced configuration. This is useful when you want a dedicated cluster of 4 nodes (for example you can set the min_nodes and max_nodes to 4), or want the compute to be within an existing VNet in your subscription.

In addition to `vm_size` and `max_nodes`, you can specify:
* `min_nodes`: Minimum nodes (default 0 nodes) to downscale to while running a job on AmlCompute
* `vm_priority`: Choose between 'dedicated' (default) and 'lowpriority' VMs when provisioning AmlCompute. Low Priority VMs use Azure's excess capacity and are thus cheaper but risk your run being pre-empted
* `idle_seconds_before_scaledown`: Idle time (default 120 seconds) to wait after run completion before auto-scaling to min_nodes
* `vnet_resourcegroup_name`: Resource group of the **existing** VNet within which AmlCompute should be provisioned
* `vnet_name`: Name of VNet
* `subnet_name`: Name of SubNet within the VNet

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           vm_priority='lowpriority',
                                                           min_nodes=2,
                                                           max_nodes=4,
                                                           idle_seconds_before_scaledown='300',
                                                           vnet_resourcegroup_name='<my-resource-group>',
                                                           vnet_name='<my-vnet-name>',
                                                           subnet_name='<my-subnet-name>')
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

### Configure & Run

In [ ]:
# Set compute target to the one created in previous step
src.run_config.target = cpu_cluster.name
 
run = experiment.submit(config=src)
run

In [ ]:
%%time
# Shows output of the run on stdout.
run.wait_for_completion(show_output=True)

In [ ]:
run.get_metrics()

## Additional operations to perform on AmlCompute

You can perform more operations on AmlCompute such as updating the node counts or deleting the compute. 

In [ ]:
#Get_status () gets the latest status of the AmlCompute target
cpu_cluster.get_status().serialize()


In [ ]:
#Update () takes in the min_nodes, max_nodes and idle_seconds_before_scaledown and updates the AmlCompute target
#cpu_cluster.update(min_nodes=1)
#cpu_cluster.update(max_nodes=10)
cpu_cluster.update(idle_seconds_before_scaledown=300)
#cpu_cluster.update(min_nodes=2, max_nodes=4, idle_seconds_before_scaledown=600)

In [ ]:
#Delete () is used to deprovision and delete the AmlCompute target. Useful if you want to re-use the compute name 
#'cpu-cluster' in this case but use a different VM family for instance.

#cpu_cluster.delete()

## Success!
Great, you are ready to move on to the remaining notebooks.